# MQTT Publish 測試程式

這個 notebook 用於測試 MQTT 發布訊息功能。


In [22]:
import paho.mqtt.client as mqtt
import time
import json
from datetime import datetime


In [23]:
# MQTT Broker 設定
MQTT_BROKER = "localhost"  # 或使用 "localhost" 如果是本機
MQTT_PORT = 1883
MQTT_USERNAME = "pi"
MQTT_PASSWORD = "raspberry"
MQTT_CLIENT_ID = "test_publisher"

print(f"MQTT Broker: {MQTT_BROKER}:{MQTT_PORT}")
print(f"Username: {MQTT_USERNAME}")


MQTT Broker: localhost:1883
Username: pi


In [24]:
# 建立 MQTT 客戶端（使用新的 Callback API VERSION2）
try:
    client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2, client_id=MQTT_CLIENT_ID)
except AttributeError:
    # 如果版本較舊，使用舊的方式
    client = mqtt.Client(client_id=MQTT_CLIENT_ID)

# 設定使用者名稱和密碼
if MQTT_USERNAME and MQTT_PASSWORD:
    client.username_pw_set(MQTT_USERNAME, MQTT_PASSWORD)

# 連線回調函數
def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("✅ 成功連接到 MQTT Broker!")
    else:
        print(f"❌ 連線失敗，錯誤代碼: {rc}")

# 發布回調函數
def on_publish(client, userdata, mid):
    print(f"📤 訊息已發布 (Message ID: {mid})")

# 設定回調函數
client.on_connect = on_connect
client.on_publish = on_publish

# 如果已經連線，先斷開
try:
    client.loop_stop()
    client.disconnect()
except:
    pass

# 連接到 Broker
try:
    client.connect(MQTT_BROKER, MQTT_PORT, 60)
    client.loop_start()  # 在背景執行網路迴圈
    time.sleep(1)  # 等待連線建立
except Exception as e:
    print(f"❌ 連線錯誤: {e}")


Exception in thread paho-mqtt-client-test_publisher:
Traceback (most recent call last):
  File "/home/pi/.local/share/uv/python/cpython-3.10.19-linux-aarch64-gnu/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/pi/.local/share/uv/python/cpython-3.10.19-linux-aarch64-gnu/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/home/pi/Documents/Github/1026_raspberry_pico/.venv/lib/python3.10/site-packages/paho/mqtt/client.py", line 4523, in _thread_main
    self.loop_forever(retry_first_connection=True)
  File "/home/pi/Documents/Github/1026_raspberry_pico/.venv/lib/python3.10/site-packages/paho/mqtt/client.py", line 2297, in loop_forever
    rc = self._loop(timeout)
  File "/home/pi/Documents/Github/1026_raspberry_pico/.venv/lib/python3.10/site-packages/paho/mqtt/client.py", line 1686, in _loop
    rc = self.loop_read()
  File "/home/pi/Documents/Github/1026_raspberry_pico/.venv/lib/python3.10/site-pa

## 測試 1: 發布簡單文字訊息


In [25]:
# 發布簡單文字訊息
# 注意：發布時不能使用萬用字元 (# 或 +)，必須使用具體的主題名稱
topic = "客廳/message"  # 改為具體主題，不要使用 #
message = "Hello MQTT! 這是測試訊息"

result = client.publish(topic, message, qos=1)
if result.rc == mqtt.MQTT_ERR_SUCCESS:
    print(f"✅ 發布成功!")
    print(f"   主題: {topic}")
    print(f"   訊息: {message}")
else:
    print(f"❌ 發布失敗，錯誤代碼: {result.rc}")


✅ 發布成功!
   主題: 客廳/message
   訊息: Hello MQTT! 這是測試訊息


## 測試 2: 發布 JSON 格式訊息


In [26]:
# 發布 JSON 格式訊息
topic = "客廳/sensor"
data = {
    "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "temperature": 25.5,
    "humidity": 60.0,
    "status": "正常"
}

message = json.dumps(data, ensure_ascii=False)
result = client.publish(topic, message, qos=1)

if result.rc == mqtt.MQTT_ERR_SUCCESS:
    print(f"✅ 發布成功!")
    print(f"   主題: {topic}")
    print(f"   訊息: {message}")
else:
    print(f"❌ 發布失敗，錯誤代碼: {result.rc}")


✅ 發布成功!
   主題: 客廳/sensor
   訊息: {"timestamp": "2025-11-30 11:55:54", "temperature": 25.5, "humidity": 60.0, "status": "正常"}


## 測試 3: 連續發布多筆訊息


In [27]:
# 連續發布多筆訊息
topic = "客廳/counter"

for i in range(5):
    message = f"訊息編號: {i+1}, 時間: {datetime.now().strftime('%H:%M:%S')}"
    result = client.publish(topic, message, qos=1)
    
    if result.rc == mqtt.MQTT_ERR_SUCCESS:
        print(f"✅ [{i+1}/5] {message}")
    else:
        print(f"❌ [{i+1}/5] 發布失敗")
    
    time.sleep(1)  # 等待 1 秒


✅ [1/5] 訊息編號: 1, 時間: 11:55:54
✅ [2/5] 訊息編號: 2, 時間: 11:55:55
✅ [3/5] 訊息編號: 3, 時間: 11:55:56
✅ [4/5] 訊息編號: 4, 時間: 11:55:57
✅ [5/5] 訊息編號: 5, 時間: 11:55:58


## 測試 4: 發布保留訊息 (Retained Message)


In [28]:
# 發布保留訊息（新訂閱者會立即收到最後一則訊息）
topic = "test/retained"
message = "這是一則保留訊息，新訂閱者會立即收到"

result = client.publish(topic, message, qos=1, retain=True)

if result.rc == mqtt.MQTT_ERR_SUCCESS:
    print(f"✅ 保留訊息發布成功!")
    print(f"   主題: {topic}")
    print(f"   訊息: {message}")
    print(f"   (新訂閱此主題的客戶端會立即收到此訊息)")
else:
    print(f"❌ 發布失敗，錯誤代碼: {result.rc}")


✅ 保留訊息發布成功!
   主題: test/retained
   訊息: 這是一則保留訊息，新訂閱者會立即收到
   (新訂閱此主題的客戶端會立即收到此訊息)


## 關閉連線


In [29]:
# 關閉連線
client.loop_stop()
client.disconnect()
print("🔌 已斷開 MQTT 連線")


🔌 已斷開 MQTT 連線
